In [3]:
#python
import gc
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
#torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
#torchvision
from torchvision import datasets, transforms
import torchvision.models as models
#import custom VGG models
from . import custom_models


ImportError: cannot import name 'custom_models'

In [2]:
def get_chest_features():
  features_path = "/home/ubuntu/6.867-xray-project/Extra_Features/"
  data_path = "/storage/x-ray-data/data/" 

  train_data = np.load(features_path + "train_data.npy")
  train_labels = np.load(features_path +  "train_labels.npy")

  val_data = np.load(features_path + "val_data.npy")
  val_labels = np.load(features_path + "val_labels.npy")

  test_data = np.load(features_path + "test_data.npy")
  test_labels = np.load(features_path + "test_labels.npy")

  transform = transforms.Compose([
    transforms.ToTensor()
])
  root = data_path + "train"
  trainset = datasets.ImageFolder(root = root, transform = transform)

  root = data_path + 'val'
  valset = datasets.ImageFolder(root = root, transform = transform)

  root = data_path + 'test'
  testset = datasets.ImageFolder(root = root, transform = transform)

  train_info = []
  for i in range(len(train_labels)):
    train_info.append((trainset[i][0], train_data[i], trainset[i][1]))

  val_info = []
  for i in range(len(val_labels)):
    val_info.append((valset[i][0], val_data[i], valset[i][1]))

  test_info = []
  for i in range(len(test_labels)):
    test_info.append((testset[i][0], test_data[i], testset[i][1]))

  trainloader = torch.utils.data.DataLoader(train_info, batch_size=1, shuffle= True, num_workers=2)
  valloader = torch.utils.data.DataLoader(val_info, batch_size=1, shuffle= False, num_workers=2)
  testloader = torch.utils.data.DataLoader(test_info, batch_size=1, shuffle= False, num_workers=2)

  return trainloader, valloader, testloader

In [3]:
class Chest_Disease_Net(nn.Module):
    def __init__(self):
        num_classes = 14
        num_features= 12
        super(Chest_Disease_Net, self).__init__()
        #define CNN as VGG11
        self.cnn = models.vgg11(pretrained=False, progress = True)
        vgg_features = self.cnn.classifier[6].in_features
        self.cnn.classifier[6] = nn.Linear(vgg_features, num_classes)
        fc1_out = 32
        #define output layers
        self.fc1 = nn.Linear(num_classes + num_features, fc1_out)
        self.fc2 = nn.Linear(fc1_out, num_classes)
        
    def forward(self, image, data):
        x1 = self.cnn(image)
        x2 = data
        x = torch.cat((x1, x2), dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
def run():
    # Parameters
    num_epochs = 10

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = Chest_Disease_Net()
    model = model.to(device)

    trainloader, valloader, testloader = get_chest_features()

    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, features, labels) in enumerate(tqdm(trainloader), 1):
            inputs = inputs.to(device)
            features = feature.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs, features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print("Loss: " + str(running_loss/len(trainloader)))
        # save after every epoch
        torch.save(model.state_dict(), "model.%d" % epoch)

        model.eval()

        train_correct = 0
        train_total = 0
        with torch.no_grad():
            for data in tqdm(trainloader):
                images, features, labels = data
                images = images.to(device)
                features = feature.to(device)
                labels = labels.to(device)

                outputs = model(images, features)
                _, predicted = torch.max(outputs.data, 1)

                train_total += labels.size(0)

                train_correct += (predicted == labels).sum().item()
        print('Top One Error of the network on train images: %d %%' % (
                100 * (1 - train_correct / train_total)))


        correct = 0
        val_total = 0
        with torch.no_grad():
            for data in tqdm(valloader):
                images, features, labels = data
                images = images.to(device)
                features = feature.to(device)
                labels = labels.to(device)

                outputs = model(images, features)
                _, predicted = torch.max(outputs.data, 1)

                val_total += labels.size(0)

                correct += (predicted == labels).sum().item()

        print('Top One Error of the network on validation images: %d %%' % (
                100 * (1 - correct / val_total)))

        gc.collect()
run()